### This notebook is intended for demonstration purposes only. 
The code might not be maintained

In [1]:
import os
os.chdir('..')

In [2]:
import json
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold

from sklearn.utils.class_weight import compute_class_weight
import random
from tensorflow.keras.regularizers import l2

from numpy.random import seed
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from datetime import datetime
from sklearn.preprocessing import StandardScaler

seed(42)
tf.random.set_seed(42)
random.seed(42)


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Dense, Flatten, Conv1D, Dropout, BatchNormalization, MaxPooling1D, LeakyReLU

class Base(Layer):
    def __init__(self, h_params, nb_classes, batch_size, variant, activation):
        super(Base, self).__init__()

        self.model = None
        self.__previous_models = None
        self.__input_shape = None

        self.__h_params = h_params
        self.__activation = activation
        self.__variant = variant
        self.__nb_classes = nb_classes
        self.__batch_size = batch_size

    def build(self, input_shape):
        exit('The function `build` needs to be implemented. This is an abstract class.')

    def get_model_name(self):
        exit('`get_model_name` needs to be implemented. This is an abstract class.')

    @property
    def variant(self):
        return self.__variant

    @property
    def batch_size(self):
        return self.__batch_size

    @property
    def nb_classes(self):
        return self.__nb_classes

    @property
    def input_shape(self):
        return self.__input_shape

    @property
    def activation(self):
        return self.__activation

    @property
    def h_params(self):
        return self.__h_params

    @property
    def n_epochs(self):
        return self.__h_params['n_epochs']

    @property
    def wd(self):
        return self.__h_params['wd']

    @property
    def l1(self):
        return self.__h_params['l1']

    @variant.setter
    def variant(self, variant):
        self.__variant = variant

    @batch_size.setter
    def batch_size(self, batch_size):
        self.__batch_size = batch_size

    @nb_classes.setter
    def nb_classes(self, nb_classes):
        self.__nb_classes = nb_classes

    @input_shape.setter
    def input_shape(self, input_shape):
        self.__input_shape = input_shape

    @activation.setter
    def activation(self, activation):
        self.__activation = activation

    @h_params.setter
    def h_params(self, h_params):
        self.__h_params = h_params

    @n_epochs.setter
    def n_epochs(self, n_epochs):
        self.__h_params['n_epochs'] = n_epochs

    @wd.setter
    def wd(self, wd):
        self.__h_params['wd'] = wd

    @l1.setter
    def l1(self, l1):
        self.__h_params['l1'] = l1


class CNN(Base):
    def __init__(self, h_params, nb_classes, batch_size, variant='lecun', activation='relu'):
        super(CNN, self).__init__(h_params, nb_classes, batch_size, variant, activation)

    def build(self, input_shape):
        self.input_shape = input_shape
        if self.variant == 'lecun':
            self.lecun()
        elif self.variant == 'lenet':
            self.lenet()
        elif self.variant == 'vgg9':
            self.vgg9()
        else:
            exit(f'Model {self.variant} unrecognized.\n Accepted variants: lecun, lenet and vgg9')

    def get_model_name(self):
        return "CNN"

    def lecun(self):
        try:
            assert self.nb_classes is not None
        except ValueError:
            exit("Must set the number of classes first")
        self.model = Sequential([
            Conv1D(filters=6, kernel_size=21, strides=1, padding='same', activation='relu',
                   input_shape=self.input_shape, kernel_initializer=keras.initializers.he_normal(), activity_regularizer=l1_l2(self.l1, self.wd)),
            BatchNormalization(),
            MaxPooling1D(pool_size=2, strides=2, padding='same'),
            Conv1D(filters=16, kernel_size=5, strides=1, padding='same', activation='relu', activity_regularizer=l1_l2(self.l1, self.wd)),
            BatchNormalization(),
            MaxPooling1D(pool_size=2, strides=2, padding='same'),
            Flatten(),
            Dense(120, activation='relu', activity_regularizer=l1_l2(self.l1, self.wd)),
            Dense(84, activity_regularizer=l1_l2(self.l1, self.wd)),
            Dense(self.nb_classes, activation='softmax', activity_regularizer=l1_l2(self.l1, self.wd))  # or Activation('softmax')
        ])

    def lenet(self):
        self.model = Sequential([
            Conv1D(filters=16, kernel_size=21, strides=1, padding='same', input_shape=self.input_shape,
                   kernel_initializer=keras.initializers.he_normal(), activity_regularizer=l1_l2(self.l1, self.wd)),
            BatchNormalization(),
            LeakyReLU(),
            MaxPooling1D(pool_size=2, strides=2, padding='same'),
            Conv1D(filters=32, kernel_size=11, strides=1, padding='same', activity_regularizer=l1_l2(self.l1, self.wd)),
            BatchNormalization(),
            LeakyReLU(),
            MaxPooling1D(pool_size=2, strides=2, padding='same'),
            Conv1D(filters=64, kernel_size=5, strides=1, padding='same', activity_regularizer=l1_l2(self.l1, self.wd)),
            BatchNormalization(),
            LeakyReLU(),
            MaxPooling1D(pool_size=2, strides=2, padding='same'),
            Flatten(),
            Dense(2050, activation='relu', activity_regularizer=l1_l2(self.l1, self.wd)),
            Dropout(0.5),
            Dense(self.nb_classes, activation='softmax', activity_regularizer=l1_l2(self.l1, self.wd))  # or Activation('softmax')
        ])

    def vgg9(self):
        self.model = Sequential([
            Conv1D(filters=64, kernel_size=21, strides=1, padding='same', activation='relu',
                   input_shape=self.input_shape, kernel_initializer=keras.initializers.he_normal(), activity_regularizer=l1_l2(self.l1, self.wd)),
            BatchNormalization(),
            Conv1D(filters=64, kernel_size=21, strides=1, padding='same', activation='relu', activity_regularizer=l1_l2(self.l1, self.wd)),
            BatchNormalization(),
            MaxPooling1D(pool_size=2, strides=2, padding='same'),
            Conv1D(filters=128, kernel_size=11, strides=1, padding='same', activation='relu', activity_regularizer=l1_l2(self.l1, self.wd)),
            BatchNormalization(),
            Conv1D(filters=128, kernel_size=11, strides=1, padding='same', activation='relu', activity_regularizer=l1_l2(self.l1, self.wd)),
            BatchNormalization(),
            MaxPooling1D(pool_size=2, strides=2, padding='same'),
            Conv1D(filters=256, kernel_size=5, strides=1, padding='same', activation='relu', activity_regularizer=l1_l2(self.l1, self.wd)),
            BatchNormalization(),
            Conv1D(filters=256, kernel_size=5, strides=1, padding='same', activation='relu', activity_regularizer=l1_l2(self.l1, self.wd)),
            BatchNormalization(),
            MaxPooling1D(pool_size=2, strides=2, padding='same'),
            Flatten(),
            Dense(4096, activation='relu', activity_regularizer=l1_l2(self.l1, self.wd)),
            Dropout(0.5),
            Dense(4096, activation='relu', activity_regularizer=l1_l2(self.l1, self.wd)),
            Dropout(0.5),
            Dense(self.nb_classes, activation='softmax', activity_regularizer=l1_l2(self.l1, self.wd))  # or Activation('softmax')
        ])


In [4]:
def ms_data(fname):
    from sklearn.preprocessing import minmax_scale
    mat_data = pd.read_csv(fname)
    labels = mat_data.index.values
    categories = [int(lab.split('_')[1]) for lab in labels]
    labels = [lab.split('_')[0] for lab in labels]
    mat_data = np.asarray(mat_data)
    mat_data = minmax_scale(mat_data, axis=0, feature_range=(0, 1))
    mat_data = mat_data.astype("float32")
    return mat_data, labels, categories



### Matthews Correlation Coefficient  (MCC)
<br><br>
$MCC = \dfrac{TP \times TN - FP \times FN}{\sqrt{(TP + FP)(TP + FN)(TN + FP)(TN + FN)}} $

$N = TN + TP + FN + FP$
<br><br>
$S = \dfrac{TP + FN}{N}$
<br><br>
$P = \dfrac{TP + FP}{N}$
<br><br>
$MCC = \dfrac{TP/N - S \times P}{\sqrt{PS(1 - S)(1 - P)}}$

## Functions to compute MCC, sensitivity and specificity

In [7]:
from tensorflow.keras import backend as K

# TODO use keras_confusion_matrix for everything
from sklearn.metrics import confusion_matrix

def keras_confusion_matrix(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)
    
    return tp, tn, fp, fn

#  matthews correlation coefficient
def mcc(y_true, y_pred):
    tp, tn, fp, fn = keras_confusion_matrix(y_true, y_pred)
    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

def sensitivity(y_true, y_pred):
    tp, _, fp, fn = confusion_matrix(y_true, y_pred).ravel()
    return tp / (tp + fn)

def specificity(y_true, y_pred):
    tp, tn, fp, _ = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)



## Save logs with Tensorboard

### Configuration of the hyperparameters and metrics to log with tensorboard  

In [8]:
from tensorboard.plugins.hparams import api as hp
class TensorboardLogging:

    def __init__(self, hparams_filepath, params):
        self.params = params
        self.hparams_filepath = hparams_filepath
        HP_EPOCHS = hp.HParam('epochs', hp.IntInterval(1, 50))
        HP_LR = hp.HParam('lr', hp.RealInterval(1e-6, 1e-3))
        HP_WD = hp.HParam('wd', hp.RealInterval(1e-8, 1e-3))
        HP_BS = hp.HParam('bs', hp.IntInterval(1, 512))
        HPARAMS = [HP_EPOCHS, HP_LR, HP_WD, HP_BS]
        with tf.summary.create_file_writer(hparams_filepath).as_default():
            hp.hparams_config(
                hparams=HPARAMS,
                metrics=[
                    hp.Metric('train_accuracy', display_name='Train Accuracy'),
                    hp.Metric('valid_accuracy', display_name='Valid Accuracy'),
                    hp.Metric('test_accuracy', display_name='Test Accuracy'),
                    hp.Metric('train_loss', display_name='Train Loss'),
                    hp.Metric('valid_loss', display_name='Valid Loss'),
                    hp.Metric('test_loss', display_name='Test Loss'),
                    hp.Metric('train_mcc', display_name='Train MCC'),
                    hp.Metric('valid_mcc', display_name='Valid MCC'),
                    hp.Metric('test_mcc', display_name='Test MCC')
                ],
            )

    def logging(self, traces):
        epochs = self.params['n_epochs']
        lr = self.params['lr']
        wd = self.params['wd']
        bs = self.params['bs']
        l1 = self.params['l1']
        with tf.summary.create_file_writer(self.hparams_filepath).as_default():
            hp.hparams({
                'epochs': epochs,
                'lr': lr,
                'wd': wd,
                'bs': bs,
                'l1': l1,
            })  # record the values used in this trial
            tf.summary.scalar('train_accuracy', np.mean([np.mean(x) for x in traces['train']['accuracies']]), step=1)
            tf.summary.scalar('valid_accuracy', np.mean(traces['valid']['accuracies']), step=1)
            tf.summary.scalar('test_accuracy', np.mean(traces['test']['accuracies']), step=1)
            tf.summary.scalar('train_loss', np.mean([np.mean(x) for x in traces['train']['losses']]), step=1)
            tf.summary.scalar('valid_loss', np.mean(traces['valid']['losses']), step=1)
            tf.summary.scalar('test_loss', np.mean(traces['test']['losses']), step=1)
            tf.summary.scalar('train_mcc', np.mean([np.mean(x) for x in traces['train']['mccs']]), step=1)
            tf.summary.scalar('valid_mcc', np.mean(traces['valid']['mccs']), step=1)
            tf.summary.scalar('test_mcc', np.mean(traces['test']['mccs']), step=1)



The following function is used to log the results with Tensorboard for each hyperparameters combinations in the training loop

# Table

In [9]:
df = pd.read_excel('D:\\workbench\\data\\spectro\\Canis\\mmc2.xlsx')
df = df[df['Unnamed: 1'].notna()]
df

,Lab number,Unnamed: 1,OCR number,Reception date,Tissue nature,Subtype,Pathological type,Classification,Adicap code,Unnamed: 9,Unnamed: 10,Breed,Age,Notes
0,Ds,1.0,H15-3523,2016-05-12,Articular capsule of the knee,NaN,Myxosarcoma,Grade I,OH,TZ,X7K0,Cavalier King charles,6.0,Right back limb
1,Ds,2.0,H15-3263,2016-05-12,Superficial dermis,NaN,Fibrosarcoma,Grade III,OH,TZ,X7K0,Cane Corso,8.0,"Perianal mass, invading the sub-cutaneous tissues"
2,Ds,3.0,H15-2015,2016-05-12,Subcutaneous tissue,NaN,Hemangiopericytoma,Grade I,OH,TZ,X7K0,Golder Retriever,10.0,NaN
3,Ds,4.0,H15-1570,2016-05-12,Subcutaneous tissue,NaN,MPNST,Grade II,OH,TZ,X7K0,Crossbreed,10.0,"Left axillary region. ""Malignant Peripheral Ne..."
4,Ds,5.0,H15-1480,2016-05-12,Subcutaneous tissue,NaN,Fibrosarcoma,Grade II,OH,TZ,X7K0,Beagle,10.0,Right back limb
5,Ds,6.0,H15-1457,2016-05-12,Mucosal chorion,NaN,Fibrosarcoma,Grade II,OH,OT,X7K0,Golder Retriever,7.0,"Labial mass, infiltrating the underneath muscu..."
6,Ds,7.0,H15-1289,2016-05-12,NaN,NaN,Fibrosarcoma,Grade II,OH,OT,X7K0,Golder Retriever,12.0,Shoulder mass. Recurrence. May be a MPNST
7,Ds,8.0,H15-1234,2016-05-12,Bone,Osteoblastic,Osteosarcoma,NaN,BH,LO,Q7A0,Bouledogue Français,9.0,Mandibular bone. Infiltrating the mucosal chorion
8,Ds,9.0,H15-1048,2016-05-12,Lung,NaN,Indifferenciated,Grade III,OH,RP,X7K0,Shetland,8.0,90% Necrosis so difficult to diagnose
9,Ds,10.0,H15-0037,2016-05-12,Mesenchymal tissue,NaN,Fibrosarcoma,Grade II,OH,OT,X7K0,Berger Allemand,12.0,Recurrence. Infiltrating the surrounding soft ...


In [25]:
data, labels, samples = ms_data('data/canis_intensities.csv')
len(samples)

In [27]:
labels_dict = {}
for label in labels:
    if label not in labels_dict:
        labels_dict[label] = 1
    else:
        labels_dict[label] += 1
labels_dict

{'Myxosarcoma': 60,
 'Fibrosarcoma': 473,
 'Indifferenciated': 376,
 'Rhabdomyosarcoma': 66,
 'Osteosarcoma': 339,
 'Hemangiopericytoma': 134,
 'splenic sarcoma': 63,
 'Histiocytic sarcoma': 105,
 'GIST': 70,
 'Normal': 482,
 'MPNST': 60}

In [32]:
samples_dict = {}
for label, sample in zip(labels, samples):
    name = f"{label}_{sample}"
    if name not in samples_dict:
        samples_dict[name] = 1
    else:
        samples_dict[name] += 1
samples_dict

{'Myxosarcoma_1': 60,
 'Fibrosarcoma_10': 71,
 'Indifferenciated_11': 78,
 'Rhabdomyosarcoma_12': 66,
 'Osteosarcoma_13': 69,
 'Indifferenciated_14': 72,
 'Osteosarcoma_15': 65,
 'Osteosarcoma_16': 71,
 'Hemangiopericytoma_17': 76,
 'Osteosarcoma_18': 71,
 'splenic sarcoma_19': 63,
 'Fibrosarcoma_2': 65,
 'Histiocytic sarcoma_20': 105,
 'Fibrosarcoma_21': 64,
 'Indifferenciated_22': 99,
 'Fibrosarcoma_23': 69,
 'GIST_24': 70,
 'Indifferenciated_25': 60,
 'Normal_26': 68,
 'Normal_27': 73,
 'Normal_28': 60,
 'Normal_29': 60,
 'Hemangiopericytoma_3': 58,
 'Normal_30': 60,
 'Normal_31': 46,
 'Normal_32': 60,
 'Normal_33': 55,
 'MPNST_4': 60,
 'Fibrosarcoma_5': 64,
 'Fibrosarcoma_6': 62,
 'Fibrosarcoma_7': 78,
 'Osteosarcoma_8': 63,
 'Indifferenciated_9': 67}

In [11]:
if 'Normal' in labels:
    for i, label in enumerate(labels):
        if label != 'Normal':
            labels[i] = 'Not Normal'

categories = pd.Categorical(labels).codes
labels = pd.concat([
    pd.DataFrame(np.array(samples).reshape([-1, 1])),
    pd.DataFrame(np.array(categories).reshape([-1, 1])),
    pd.DataFrame(np.array(labels).reshape([-1, 1])),
], 1)
labels.columns = ['sample', 'category', 'label']
labels

,sample,category,label
0,1,1,Not Normal
1,1,1,Not Normal
2,1,1,Not Normal
3,1,1,Not Normal
4,1,1,Not Normal
...,...,...,...
2223,9,1,Not Normal
2224,9,1,Not Normal
2225,9,1,Not Normal
2226,9,1,Not Normal


# How many sample per class is there really?

In [12]:
set(labels['sample'])

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33}

In [13]:
set(labels['label'])

{'Normal', 'Not Normal'}

In [14]:
set(labels['category'])

{0, 1}

In [15]:
class Train:
    def __init__(
            self,
            intensities_file,
            cumulative_step,
            criterion='categorical_crossentropy',
            variant='logistic',
            get_data_function=ms_data,
            n_channels=1,
            save_train_models=True,
            model_name=CNN,
            verbose=0,
            model_path=False,
            freeze=True,
            retrain=False,
    ):
        self.dataset_name = intensities_file.split('/')[-1].split('_')[0]
        self.freeze = freeze
        self.verbose = verbose
        self.variant = variant
        self.retrain = retrain
        self.cumulative_step = cumulative_step
        self.model_name = model_name
        self.save_train_models = save_train_models
        self.data, labels, samples = get_data_function(intensities_file)
        self.data[np.isnan(self.data)] = 0

        # TODO have this step done in R import_data.R

        if 'Normal' in labels:
            for i, label in enumerate(labels):
                if label != 'Normal':
                    labels[i] = 'Not Normal'

        categories = pd.Categorical(labels).codes
        self.labels = pd.concat([
            pd.DataFrame(np.array(samples).reshape([-1, 1])),
            pd.DataFrame(np.array(categories).reshape([-1, 1])),
            pd.DataFrame(np.array(labels).reshape([-1, 1])),
        ], 1)
        self.labels.columns = ['sample', 'category', 'label']
        self.nb_classes = len(np.unique(self.labels['category']))
        self.input_shape = [self.data.shape[1], n_channels]
        self.criterion = criterion
        self.step = 0
        self.call_num = 0
        self.previous_datasets = ""
        if model_path != 'None':
            self.model_path, self.previous_datasets = self.select_best_model(model_path)
        else:
            self.model_path = False
        if self.previous_datasets == "":
            self.datasets = self.dataset_name
        else:
            self.datasets = f"{self.previous_datasets}_{self.dataset_name}"
        self.params = {}

    # This function of class Train is not used to train the first model and is ommitted to simplify the code. 
    # Implemented and used notebook train_second_model (long).ipynb
    def select_best_model(self, params_fname):
        pass
    
    # This function of class Train is not used to train the first model and is ommitted to simplify the code. 
    # Implemented and used notebook train_second_model (long).ipynb
    def update_model(self, model_source, path, wd):
        pass
    
    
    def train(self, h_params):
        self.call_num += 1
        n_epochs = h_params[0]
        lr = h_params[1]
        wd = h_params[2]
        l1 = h_params[3]
        bs = h_params[4]

        h_params = {
            "n_epochs": n_epochs,
            "lr": lr,
            "wd": wd,
            "l1": l1,
            "bs": bs,
        }

        path = \
            f'{self.criterion}/' + \
            f'{n_epochs}/' \
            f'{"{:.8f}".format(float(lr))}/' \
            f'{"{:.8f}".format(float(wd))}/' \
            f'{bs}/'
        dnn = self.model_name(h_params, self.nb_classes, variant=self.variant, activation='relu', batch_size=bs)
        hparams_filepath = f"logs/{dnn.get_model_name()}/{self.variant}/{self.datasets}/{self.freeze}/{self.retrain}" \
                           f"/hparam_tuning/{path}"
        log_filepath = f"logs/{dnn.get_model_name()}/{self.variant}/{self.datasets}/{self.freeze}/{self.retrain}/{path}"
        del dnn
        os.makedirs(log_filepath, exist_ok=True)
        os.makedirs(hparams_filepath, exist_ok=True)
        tb_logging = TensorboardLogging(hparams_filepath, h_params)

        traces = {
            "train": {
                "losses": [],
                "accuracies": [],
                "mccs": [],
                "sensitivities": [],
                "specificities": [],
            },
            "valid": {
                "losses": [],
                "accuracies": [],
                "mccs": [],
                "sensitivities": [],
                "specificities": [],
            },
            "test": {
                "losses": [],
                "accuracies": [],
                "mccs": [],
                "sensitivities": [],
                "specificities": [],
            },
        }
        
        ################################################################
        # BEGINNING OF PART DIFFERENT FROM CORRECT SPLIT
        # Duplicates of the same biological sample will end up 
        # in all train/valid/test splits and lead to overfitting
        ################################################################
        
        indices = np.array(list(range(len(self.labels['category']))))
        np.random.shuffle(indices)
        cutoff = int(len(indices) * 0.8)
        all_train_indices = indices[:cutoff]
        all_train_labels = self.labels['category'][all_train_indices].tolist()
        test_indices = indices[cutoff:]
        x_test = self.data[test_indices]
        y_test = self.labels['category'][test_indices].tolist()

        x_test = self.data[test_indices]
        y_test = self.labels['category'][test_indices].tolist()

        assert len(set(y_test)) == self.nb_classes

        # 3 Fold-CV
        skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
        for i, (train_samples, valid_samples) in enumerate(skf.split(all_train_samples, train_cats)):
            # Just plot the first iteration, it will already be crowded if doing > 100 optimization iterations
            if self.verbose:
                print(f"CV: {i}")
            # new_nums = [train_nums[i] for i in inds]

            valid_samples = [all_train_samples[s] for s in valid_samples]
            train_samples = [all_train_samples[s] for s in train_samples]
            train_indices = [s for s, lab in enumerate(self.labels['sample'].tolist()) if lab in train_samples]
            valid_indices = [s for s, lab in enumerate(self.labels['sample'].tolist()) if lab in valid_samples]

            ################################################################
            # END OF PART DIFFERENT FROM CORRECT SPLIT
            ################################################################
            
            x_train = self.data[train_indices]
            y_train = self.labels['category'][train_indices]
            x_valid = self.data[valid_indices]
            y_valid = self.labels['category'][valid_indices]

            assert len(set(y_train)) == self.nb_classes and len(set(y_valid)) == self.nb_classes
            scaler = StandardScaler()
            scaler.fit(x_train)
            x_train = scaler.transform(x_train)
            x_valid = scaler.transform(x_valid)

            x_train_conv = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
            x_valid_conv = np.reshape(x_valid, (x_valid.shape[0], x_valid.shape[1], 1))
            y_train_conv = to_categorical(y_train, self.nb_classes)
            y_valid_conv = to_categorical(y_valid, self.nb_classes)

            dnn = self.model_name(h_params, self.nb_classes, batch_size=bs, variant=self.variant, activation='relu')
            dnn.build(input_shape=self.input_shape)

            if self.model_path:
                dnn = self.update_model(model_source=dnn, path=self.model_path, wd=wd)
            dnn.model.compile(loss=self.criterion,
                              optimizer=Adam(lr=lr, beta_1=0.99, beta_2=0.999, decay=wd, amsgrad=False),
                              metrics=['accuracy', mcc])
            callbacks = []
            if i == 0:
                callbacks += [keras.callbacks.TensorBoard(
                    log_dir=log_filepath,
                    histogram_freq=1,
                    write_graph=True,
                    write_images=False,
                    update_freq="epoch",
                    profile_batch=2,
                    embeddings_freq=0,
                    embeddings_metadata=None,
                )]
            callbacks += [keras.callbacks.EarlyStopping(
                monitor='loss',
                min_delta=0,
                patience=10,
                verbose=self.verbose,
                mode='min'
            )]

            y_integers = np.argmax(y_train_conv, axis=1)
            class_weights = compute_class_weight('balanced', classes=np.unique(y_integers), y=y_integers)
            d_class_weights = dict(enumerate(class_weights))

            # CAN't USE VALIDATION SPLIT, so also can't use early_stopping or reduceLROnPLateau
            if self.verbose == 2:
                fit_verbose = 1
            else:
                fit_verbose = 0
            history = dnn.model.fit(
                x=x_train_conv,
                y=y_train_conv,
                batch_size=bs,
                verbose=fit_verbose,
                epochs=n_epochs,
                validation_split=0.,
                class_weight=d_class_weights,
                callbacks=callbacks
            )
            base_path = f'saved_models/keras/{dnn.get_model_name()}/{self.variant}/{self.datasets}/' \
                        f'{self.freeze}/{self.retrain}/'
            file_path = f'{base_path}/{path}'
            os.makedirs(file_path, exist_ok=True)

            train_acc = history.history['accuracy']
            train_loss = history.history['loss']
            train_mcc = history.history['mcc']
            train_sensitivity = history.history['mcc']
            train_mcc = history.history['mcc']

            # train_loss, train_acc, train_mcc = dnn.model.evaluate(x_train_conv, y_train_conv, verbose=self.verbose)
            y_classes_train = np.argmax(dnn.model.predict(x_train_conv), axis=-1)
            train_sensitivity = sensitivity(y_train, y_classes_train)
            train_specificity = specificity(y_train, y_classes_train)
            
            valid_loss, valid_acc, valid_mcc = dnn.model.evaluate(x_valid_conv, y_valid_conv, verbose=self.verbose)
            y_classes_valid = np.argmax(dnn.model.predict(x_valid_conv), axis=-1)
            valid_sensitivity = sensitivity(y_valid, y_classes_valid)
            valid_specificity = specificity(y_valid, y_classes_valid)

            best_epoch = np.argmin(valid_loss)

            traces['train']['losses'].append(train_loss)
            traces['train']['accuracies'].append(train_acc)
            traces['train']['mccs'].append(train_mcc)
            traces['train']['sensitivities'].append(train_sensitivity)
            traces['train']['specificities'].append(train_specificity)
            traces['valid']['losses'].append(valid_loss)
            traces['valid']['accuracies'].append(valid_acc)
            traces['valid']['mccs'].append(valid_mcc)
            traces['valid']['sensitivities'].append(valid_sensitivity)
            traces['valid']['specificities'].append(valid_specificity)

        if self.retrain:
            x_all_train = self.data[all_train_indices]
            y_all_train = self.labels['category'][all_train_indices]

            scaler = StandardScaler()
            scaler.fit(x_all_train)
            all_x_train = scaler.transform(x_all_train)

            x_train_conv = np.reshape(all_x_train, (all_x_train.shape[0], all_x_train.shape[1], 1))
            y_train_conv = to_categorical(y_all_train, self.nb_classes)

            history = dnn.model.fit(
                x=x_train_conv,
                y=y_train_conv,
                batch_size=bs,
                verbose=fit_verbose,
                epochs=n_epochs,
                validation_split=0.,
                class_weight=d_class_weights,
                callbacks=callbacks
            )

        x_test_trans = scaler.transform(x_test)
        x_test_conv = np.reshape(x_test_trans, (x_test_trans.shape[0], x_test_trans.shape[1], 1))
        y_test_conv = to_categorical(y_test, self.nb_classes)

        test_loss, test_acc, test_mcc = dnn.model.evaluate(x_test_conv, y_test_conv, verbose=self.verbose)
        y_classes_test = np.argmax(dnn.model.predict(x_test_conv), axis=-1)
        test_sensitivity = sensitivity(y_test, y_classes_test)
        test_specificity = specificity(y_test, y_classes_test)
        
        traces['test']['losses'].append(test_loss)
        traces['test']['accuracies'].append(test_acc)
        traces['test']['mccs'].append(test_mcc)
        traces['test']['specificities'].append(test_specificity)
        traces['test']['sensitivities'].append(test_sensitivity)
        # dnn.model.summary()

        dnn.model.save_weights(f'{file_path}/{self.variant}_{self.cumulative_step}.h5')
        self.step += 1

        tb_logging.logging(traces)

        self.params[f"call_{self.call_num}"] = {
            'datasets': self.datasets,
            'h_params': {
                'criterion': f'{self.criterion}',
                'n_epochs': f'{n_epochs}',
                'lr': f'{lr}',
                'bs': f'{bs}',
                'wd': f'{wd}',
                'l1': f'{l1}',
            },
            'scores': {
                'best_epoch': f'{best_epoch}',
                'train_loss': f'{train_loss[int(best_epoch)]}',
                'valid_loss': f'{valid_loss}',
                'test_loss': f'{test_loss}',
                'train_acc': f'{train_acc[int(best_epoch)]}',
                'valid_acc': f'{valid_acc}',
                'test_acc': f'{test_acc}',
                'train_mcc': f'{train_mcc[int(best_epoch)]}',
                'valid_mcc': f'{valid_mcc}',
                'test_mcc': f'{test_mcc}',
            }
        }
        json.dump(self.params, open(f'{base_path}/{self.freeze}/{self.retrain}/params.json', 'w'))

        return 1 - np.mean(traces['valid']['accuracies'])

    def test(self, params):
        pass



Use the following cell to launch tensorboard directly in the notebook. <br>
If you have trouble lauching the tensorboard from the notebook, try launching it from a terminal with the command:<br>
`tensorboard --logdir logs` <br>
It is easier to use a dedicated tab to see the results anyways.

In [16]:
%load_ext tensorboard
%tensorboard --logdir logs/CNN/lecun/canis


Reusing TensorBoard on port 6006 (pid 9632), started 3:39:25 ago. (Use '!kill 9632' to kill it.)

In [17]:
# Verify the gpu is used. If empty list, no GPU available
import tensorflow as tf 
tf.config.list_physical_devices('GPU')

[]

In [18]:
train = Train(intensities_file='data/canis_intensities.csv',
              cumulative_step=0,
              criterion= 'categorical_crossentropy',
              variant='lecun',
              verbose=1,
              model_name=CNN,
              model_path='None',
              freeze=True,
              retrain=True
              )

os.makedirs(f'logs/hparam_tuning', exist_ok=True)

from skopt.space import Real, Integer
from skopt import gp_minimize

space = [
    Integer(1, 50, "uniform", name='epochs'),
    Real(1e-6, 1e-3, "log-uniform", name='lr'),
    Real(1e-8, 1e-3, "log-uniform", name='wd'),
    Real(1e-8, 1e-3, "log-uniform", name='l1'),
    Integer(1, 512, "uniform", name='bs'),
]

test_mean = gp_minimize(train.train, space, n_calls=100, random_state=42)
test_mean

CV: 0


KeyboardInterrupt: 

In [ ]:
def select_best_model(params_fname):
    # import from json file
    params = pd.read_json(params_fname)
    best_call = 0
    best_valid_loss = np.inf
    for call in list(params.keys()):
        values = params[call]
        if float(values['scores']['valid_loss']) < best_valid_loss:
            best_valid_loss = float(values['scores']['valid_loss'])
            best_call = call
    hparams = params[best_call]['h_params']

    return hparams

# True and True for freezing the weights and retrain after cross-validation.
# TODO save with meaningful folder names
select_best_model("saved_models/keras/CNN/lecun/canis/True/True/params.json")

To see the results, use tensorboard or (not recommended) change the verbose option to 1.